In [1]:
try:
    import deepmatcher as dm
except:
    !pip install -qqq deepmatcher

     |████████████████████████████████| 51kB 2.0MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 194kB 9.9MB/s 
  ERROR: Failed building wheel for fasttextmirror
    Running setup.py install for fasttextmirror ... done


In [2]:
import deepmatcher as dm
import torch
torch.cuda.is_available()
import pandas as pd
import io
from google.colab import files
from google.colab import drive

In [3]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip --directory-prefix=/root/.vector_cache
!unzip /root/.vector_cache/wiki.en.zip -d /root/.vector_cache/
!rm /root/.vector_cache/wiki.en.vec

--2021-02-23 17:55:40--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10356881291 (9.6G) [application/zip]
Saving to: ‘/root/.vector_cache/wiki.en.zip’

wiki.en.zip         100%[===================>]   9.65G  50.0MB/s    in 3m 28s  

2021-02-23 17:59:08 (47.5 MB/s) - ‘/root/.vector_cache/wiki.en.zip’ saved [10356881291/10356881291]

Archive:  /root/.vector_cache/wiki.en.zip
  inflating: /root/.vector_cache/wiki.en.vec  
  inflating: /root/.vector_cache/wiki.en.bin  


In [4]:
from google.colab import files

uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv
Saving validate.csv to validate.csv


In [5]:
train, validation, test = dm.data.process(
    path='',
    train='train.csv',
    validation='validate.csv',
    test='test.csv')


Reading and processing data from "train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "validate.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#############] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01

Computing principal components
0% [#############] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


In [6]:
train_table = train.get_raw_table()
train_table.head()
df = train_table[train_table["label"] == 1]
df.head()



,id,label,left_id,left_name,left_host_id,left_host_name,left_neighbourhood_group,left_neighbourhood,left_latitude,left_longitude,left_room_type,left_price,left_minimum_nights,left_number_of_reviews,left_last_review,left_reviews_per_month,right_id,right_name,right_host_id,right_host_name,right_neighbourhood_group,right_neighbourhood,right_latitude,right_longitude,right_room_type,right_price,right_minimum_nights,right_number_of_reviews,right_last_review,right_reviews_per_month
1,1617,1,17931967,bigfamilyflats - sistine chapel - 18 pax - net...,6146778,paolo & friends,,i centro storico,41.8996,12.46203,entire home/apt,6658,2,111,2020-02-27,2.4,18333504,the dome stay- amazing view on the river - net...,6146778,paolo & friends,,i centro storico,41.8996,12.46203,private room,205,1,101,2020-08-30,2.23
13,786,1,8998658,"central brand new apartment , two bedroom",42973707,riccardo,,vii san giovanni/cinecittà,41.87294,12.526489999999999,entire home/apt,91,2,9,2019-06-03,0.15,24052365,comfort apartment 2 rooms + private bathrooms,42973707,riccardo,,vii san giovanni/cinecittà,41.87294,12.526489999999999,entire home/apt,91,2,2,2019-11-22,0.06
15,3017,1,32007919,smart trastevere,235391429,daniel,,xii monte verde,41.87547,12.46605,private room,59,1,6,2019-10-06,0.28,32567054,smart trastevere,235391429,daniel,,xii monte verde,41.87547,12.46605,shared room,15,1,22,2020-02-22,0.96
17,1615,1,17931967,bigfamilyflats - sistine chapel - 18 pax - net...,6146778,paolo & friends,,i centro storico,41.8996,12.46203,entire home/apt,6658,2,111,2020-02-27,2.4,18001865,gianicolo stay room-bambin gesù /also ★long term★,6146778,paolo & friends,,i centro storico,41.8996,12.46203,private room,205,1,96,2020-03-01,2.09
23,829,1,9656868,fiftynine appartamento superior caravaggio,35397970,giulia,,i centro storico,41.90704,12.47686,entire home/apt,66,1,10,2019-06-14,0.19,15242992,fiftynine appartamento borromini,35397970,giulia,,i centro storico,41.90704,12.47686,entire home/apt,66,1,3,2018-04-22,0.07


In [7]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [8]:
model.run_train(
    train,
    validation,
    epochs=1,
    batch_size=16,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 30939816
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2398: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:18:56


Finished Epoch 1 || Run Time: 1130.4 | Load Time:    7.8 || F1:  94.83 | Prec:  91.74 | Rec:  98.14 || Ex/s:  10.88

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:02:02


Finished Epoch 1 || Run Time:  121.5 | Load Time:    2.7 || F1:  99.88 | Prec:  99.77 | Rec: 100.00 || Ex/s:  33.25

* Best F1: tensor(99.8840)
Saving best model...
Done.
---------------------

Loading best model...
Training done.


tensor(99.8840)

In [9]:
model.run_eval(test)

===>  EVAL Epoch 1


0% [██████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:32


Finished Epoch 1 || Run Time:   90.5 | Load Time:    2.4 || F1:  99.57 | Prec:  99.14 | Rec: 100.00 || Ex/s:  44.48



tensor(99.5666)

In [10]:
model.run_prediction(test)

===>  PREDICT Epoch 1


0% [██████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:35


Finished Epoch 1 || Run Time:   93.2 | Load Time:    2.4 || F1:  99.57 | Prec:  99.14 | Rec: 100.00 || Ex/s:  43.22



,match_score
id,
13460,0.130833
12605,0.136401
16912,0.135592
6610,0.135493
7272,0.126263
...,...
1244,0.975881
9269,0.138394
10418,0.144364


# Inference

Use model trained on other listing

In [11]:
model = dm.MatchingModel(attr_summarizer='hybrid') 
model.load_state("hybrid_model.pth")

In [ ]:
unlabeled = dm.data.process_unlabeled(
    path='trainRome.csv',
    trained_model=model,
    ignore_columns = "label"
)

In [ ]:
predictions = model.run_prediction(unlabeled)
predictions.head()

In [ ]:
model.run_eval(unlabeled)

In [ ]:
type(test)
test

In [ ]:

predict=model.run_prediction(test)                        

In [ ]:
predict

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:


train3, validation3, test3 = dm.data.process(
    path='',
    train='train2.csv',
    validation='validat2.csv',
    test='test2.csv', auto_rebuild_cache=True)

In [ ]:
predict2=model.run_prediction(test2) 

predict2
